In [10]:
import os

import pandas as pd

data_dir = "./data"
files = [f for f in os.listdir(data_dir)]

In [11]:
def update_fig(figure):
    figure.update_layout(
        plot_bgcolor='white',
        paper_bgcolor="white"
    )
    figure.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    figure.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    figure.update_layout(font=dict(color='#000000'))


In [12]:
file = []
res = []

for file_name in files:
    if 'final' not in file_name:
        continue

    df = pd.read_csv(data_dir + '/' + file_name)
    group_by_winner_df = df.groupby(['winner'])['winner'].size().reset_index(name='round')

    ma_win_rate = 0
    ma_avg_turn = float("INF")
    ey_win_rate = 0
    ey_avg_turn = float("INF")

    total = sum(group_by_winner_df['round'])
    if not group_by_winner_df[group_by_winner_df['winner'] == 'MARQUISE'].empty:
        ma_win = group_by_winner_df[group_by_winner_df['winner'] == 'MARQUISE'].iloc[0]['round']
        ma_win_rate = ma_win / total
        if ma_win > 0:
            ma_avg_turn = sum(df[df['winner'] == 'MARQUISE']['turn']) / len(df[df['winner'] == 'MARQUISE']['turn'])

    if not group_by_winner_df[group_by_winner_df['winner'] == 'EYRIE'].empty:
        ey_win = group_by_winner_df[group_by_winner_df['winner'] == 'EYRIE'].iloc[0]['round']
        ey_win_rate = ey_win / total
        if ey_win > 0:
            ey_avg_turn = sum(df[df['winner'] == 'EYRIE']['turn']) / len(df[df['winner'] == 'EYRIE']['turn'])

    # print(file_name)
    # print(ma_win_rate, ma_avg_turn, ey_win_rate, ey_avg_turn)
    file.append(int(file_name.split('-')[2]))
    res.append((ma_win_rate, ma_avg_turn, ey_win_rate, ey_avg_turn))

result = {file[i]: res[i] for i in range(len(file))}

In [13]:
import numpy as np

ma_model = [
    [0, 1, 2, 3, 4],
    [5, 6, 7, 8, 9],
    [10, 11, 12, 13, 14],
    [15, 16, 17, 18, 19],
    [20, 21, 22, 23, 24]
]
ma_config_name = [
    'config-86-ma',
    'config-76-ma',
    'config-85-ma',
    'config-87-ma',
    'config-89-ma'
]
ma_model_res = []

ey_model = np.array(ma_model).T.tolist()
ey_config_name = [
    'config-184-ey',
    'config-203-ey',
    'config-194-ey',
    'config-212-ey',
    'config-202-ey'
]
ey_model_res = []

for config in ma_model:
    avg_win_rate = sum([result[i][0] for i in config if i != -1]) / len([result[i][0] for i in config if i != -1])
    avg_turn_rate = sum([result[i][1] for i in config if i != -1]) / len([result[i][1] for i in config if i != -1])
    ma_model_res.append((avg_win_rate, avg_turn_rate))

for config in ey_model:
    avg_win_rate = sum([result[i][2] for i in config if i != -1]) / len([result[i][2] for i in config if i != -1])
    avg_turn_rate = sum([result[i][3] for i in config if i != -1]) / len([result[i][3] for i in config if i != -1])
    ey_model_res.append((avg_win_rate, avg_turn_rate))

print("Marquise Result")
for ind, res in enumerate(ma_model_res):
    print("{}:".format(ma_config_name[ind]))
    print("\t|_ win rate: " + str(res[0]))
    print("\t|_ avg turn: " + str(res[1]))

print("Eyrie Result")
for ind, res in enumerate(ey_model_res):
    print("{}:".format(ey_config_name[ind]))
    print("\t|_ win rate: " + str(res[0]))
    print("\t|_ avg turn: " + str(res[1]))


Marquise Result
config-86-ma:
	|_ win rate: 0.5319999999999999
	|_ avg turn: 15.362663476874005
config-76-ma:
	|_ win rate: 0.38999999999999996
	|_ avg turn: 15.727401010393171
config-85-ma:
	|_ win rate: 0.5599999999999999
	|_ avg turn: 15.389490926805504
config-87-ma:
	|_ win rate: 0.422
	|_ avg turn: 15.249598102408086
config-89-ma:
	|_ win rate: 0.374
	|_ avg turn: 15.22989010989011
Eyrie Result
config-184-ey:
	|_ win rate: 0.634
	|_ avg turn: 13.681134187732079
config-203-ey:
	|_ win rate: 0.52
	|_ avg turn: 15.387959413352073
config-194-ey:
	|_ win rate: 0.5800000000000001
	|_ avg turn: 15.217262429734443
config-212-ey:
	|_ win rate: 0.45
	|_ avg turn: 15.735108600679581
config-202-ey:
	|_ win rate: 0.538
	|_ avg turn: 15.516868508799002


In [14]:
import plotly.express as px

heatmap = [(k, v[0]) for k, v in result.items()]
heatmap_reshaped = np.reshape([v[1] for v in sorted(heatmap)], (5, 5))

fig = px.imshow(heatmap_reshaped, labels=dict(x="Eyrie", y="Marquise", color="Marquise Win Rate"), text_auto=True,
                x=ey_config_name, y=ma_config_name, zmax=1, zmin=0, color_continuous_scale='RdBu_r')
update_fig(fig)
fig.show()
fig.write_image("images/fig-stage-2-marquise-win-rate.jpeg", scale=6)

heatmap2 = [(k, v[2]) for k, v in result.items()]
heatmap_reshaped2 = np.reshape([s[1] for s in sorted(heatmap2)], (5, 5))

fig = px.imshow(heatmap_reshaped2, labels=dict(x="Eyrie", y="Marquise", color="Eyrie Win Rate"), text_auto=True,
                x=ey_config_name, y=ma_config_name, zmax=1, zmin=0, color_continuous_scale='RdBu_r')
update_fig(fig)
fig.show()
fig.write_image("images/fig-stage-2-eyrie-win-rate.jpeg", scale=6)